# Creator Metrics

This script processes a CSV from [this Dune Query](https://dune.com/queries/4087193) and derives metrics the following NFT creator metrics:

- num drops
- num unique minters
- num txns
- USD value of txns

In [1]:
from dotenv import load_dotenv
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import json
import os
import pandas as pd

2024-09-23 05:18:30,348 INFO numexpr.utils NumExpr defaulting to 8 threads.


In [2]:
load_dotenv()

DUNE_API_KEY = os.getenv('DUNE_API_KEY')
dune = DuneClient(DUNE_API_KEY)

## Part 1. Parse the creator data

In [3]:
all_apps = pd.read_csv('data/apps/applications_reviewed.csv')
app_map = all_apps.set_index('uuid')['name'].to_dict()

In [4]:
apps = pd.read_csv('data/apps/creator_apps.csv', index_col=0)
apps = apps.drop(columns=['chain']).drop_duplicates()
creator_addresses = list(apps['address'].unique())
apps_to_addresses = apps.set_index('uuid')['address'].to_dict()
apps.tail(1)

,uuid,recipient,project_type,category,address
1368,568706f3-8127-448f-b0b1-56c28e4f3ed1,0xf56E55e35d2CCa5A34F5Ba568454974424aEA0F4,Creator,Art NFTs,0xf56e55e35d2cca5a34f5ba568454974424aea0f4


In [5]:
app_dict = {}
for _,app in apps.iterrows():
    recipient = app['recipient'].lower()
    uuid = app['uuid']
    minting_address = app['address'].lower()
    if recipient not in app_dict:
        app_dict.update({
            recipient: {
                'mintingAddressList': [minting_address],
                'uuidList': [uuid]
            }
        })
    else:
        app_dict[recipient]['mintingAddressList'].append(minting_address)
        app_dict[recipient]['uuidList'].append(uuid)

app_json = []        
for recipient, record in app_dict.items():
    app_json.append({
        'recipientAddress': recipient,
        'mintingAddressList': list(set(record['mintingAddressList'])),
        'uuidList': list(set(record['uuidList'])),
    })
        
with open("data/apps/creator_address_to_uuid.json", "w") as f:
    json.dump(app_json, f, indent=2)

## Part 2. Grab NFT mint data from Dune

In [6]:
creator_addresses_str = ',\n\t\t'.join(creator_addresses)
query_sql = f"""
    select
        blockchain,
        project,
        buyer,
        tx_to,
        tx_from,
        nft_contract_address,
        project_contract_address,
        coalesce(amount_usd, 0) as amount_usd,
        number_of_items
    from nft.mints
        where
            block_date between date('2024-06-01') and date('2024-09-01')
            and blockchain in ('base', 'optimism', 'zora')
            and tx_from in  (
                {creator_addresses_str}
            )
"""

In [7]:
# query_id = dune.create_query(name='sunny_nft_mints', query_sql=query_sql, is_private=False)
# query = QueryBase(name='sunny_nft_mints', query_id=query_id.base.query_id)
# results_df = dune.run_query_dataframe(query)
# results_df.to_parquet('data/raw_metric_data/dune_raw_nft_mints.parquet')
nft_creators = pd.read_parquet('data/raw_metric_data/dune_raw_nft_mints.parquet')
nft_creators.rename(columns={'tx_from': 'creator_address'}, inplace=True)
nft_creators['blockchain'] = nft_creators['blockchain'].apply(lambda x: x.title())
nft_creators.tail()

,blockchain,project,buyer,tx_to,creator_address,nft_contract_address,project_contract_address,amount_usd,number_of_items
56075,Base,basecolors,0x00409fc839a2ec2e6d12305423d37cd011279c09,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0x00409fc839a2ec2e6d12305423d37cd011279c09,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,3.427170,1
56076,Base,Unknown,0x1a1c37c145a1eab58c43f003ebb55c18083b5987,0xad14d16d5e980900d5d87153b1de44bd8b02892d,0x1a1c37c145a1eab58c43f003ebb55c18083b5987,0xad14d16d5e980900d5d87153b1de44bd8b02892d,0xad14d16d5e980900d5d87153b1de44bd8b02892d,0.328370,1
56077,Base,basecolors,0xd91c4283ebbc00af162b73418ec4ab0b3c159900,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0xd91c4283ebbc00af162b73418ec4ab0b3c159900,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0.000000,1
56078,Base,Unknown,0x5d85e0403f815ce6b88fa7653ee84e46e5f59b95,0xfee588791cda1d01ccfc80b51efa00c0be5b129e,0x5d85e0403f815ce6b88fa7653ee84e46e5f59b95,0x1d2550d198197df1a10af515cf2ea0d790889b93,0xfee588791cda1d01ccfc80b51efa00c0be5b129e,2.771984,1
56079,Base,zora,0x4412e1fd47b780c59976694a778999698e1fe3a8,0x1bbea2cc3b2c41774687fd00684ad12f9a666557,0x589ffbbda0eacd5a9c2ba208b379c886b2630503,0x1bbea2cc3b2c41774687fd00684ad12f9a666557,0x1bbea2cc3b2c41774687fd00684ad12f9a666557,0.000000,1


In [8]:
apps.tail(1)

,uuid,recipient,project_type,category,address
1368,568706f3-8127-448f-b0b1-56c28e4f3ed1,0xf56E55e35d2CCa5A34F5Ba568454974424aEA0F4,Creator,Art NFTs,0xf56e55e35d2cca5a34f5ba568454974424aea0f4


# Part 3. Derive the metrics

In [9]:
metrics_by_address = pd.concat([
    nft_creators.groupby('creator_address')['nft_contract_address'].nunique().rename('num_drops'),
    nft_creators.groupby('creator_address')['buyer'].nunique().rename('num_unique_minters'),
    nft_creators.groupby('creator_address')['buyer'].count().rename('num_transactions'),
    nft_creators.groupby('creator_address')['amount_usd'].sum().rename('usd_value_of_transactions')
], axis=1)

In [10]:
metrics = []
errors = []
for uuid, app in apps.set_index('uuid').iterrows():
    
    name = app_map.get(uuid)
    address = app['address']
    recipient = app['recipient']
    
    if address not in metrics_by_address.index:
        errors.append({
            'uuid': uuid,
            **app,
            'status': 'no_nft_activity_90D'
        })
        continue
    
    creator_metrics = metrics_by_address.loc[address].to_dict()
    
    metrics.append({
        'uuid': uuid,
        **app,
        'status': 'metrics_available',
        **creator_metrics,
    })

In [11]:
df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv('data/clean_metric_data/metrics_creators.csv')

## Part 4. Derive metrics by recipient address

In [12]:
recipient_mapping = (
    all_apps
    .groupby(['recipient', 'address'])
    ['uuid']
    .unique()
    .apply(list)
    .to_dict()
)

In [13]:
recipient_metrics = []
for (recipient,address),uuid_list in recipient_mapping.items():
    
    dff = nft_creators[nft_creators.creator_address == address]
    if len(dff) < 1:
        continue
    
    creator_metrics = {
        'recipient': recipient,
        'minting_wallet': address,
        
        'num_drops': dff['nft_contract_address'].nunique(),
        'num_unique_minters': dff['buyer'].nunique(),
        'num_transactions': dff['buyer'].count(),
        'usd_value_of_transactions': dff['amount_usd'].sum(),
        
        'uuid_list': uuid_list
    }
    recipient_metrics.append(creator_metrics)

In [14]:
df_recipient_metrics = pd.DataFrame(recipient_metrics)
df_recipient_metrics.to_csv('data/clean_metric_data/metrics_creators_by_recipient.csv')